In [1]:
import pandas as pd
import numpy as np
import datetime as dt
path= 'z:/largetransfer/luc/carter' 

In [2]:
tci= pd.read_csv(path+ '/data/model_data/tci_1_0.csv', parse_dates=['Date'])
tci = tci[['parcel','vacant','SPA_NAME','Date']]


C:\Users\senatoduro8\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [82]:
fclose = pd.read_csv(path+'/data/clean_data/foreclosure_filings2.csv', parse_dates=[2])
fclose2=pd.read_csv(path+'/data/clean_data/foreclosure_filings.csv',parse_dates=[2],usecols=list(range(12))+[13])
transfer = pd.read_csv(path+'/data/clean_data/transfers.csv', parse_dates=['mdate'], dtype=str)
alen = pd.read_csv(path+'/data/clean_data/armslength.csv', dtype=str)
fclose=pd.merge(fclose,fclose2[['parcel','cdesig']], how='left', left_on='parcel',right_on='parcel')

In [78]:
# parse armslength dates
months = dict(zip(['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'],range(1,13)))
def parse_date(x):
    day = int(x[0:2])
    month = months[x[2:5]]
    year = int(x[5:])
    return dt.datetime(year,month,day)

alen['date']=alen.mdate.apply(parse_date)

In [83]:
##preprocessing and subsetting
transfer.columns=transfer.columns.str.lower()
alen.columns=alen.columns.str.lower()
transfer=transfer[['property_number','grantee1','mdate','deed_type']]
transfer=transfer.loc[(transfer.deed_type=='Sheriffs Deed') | (transfer.deed_type == 'Sheriffs Deed Ex'),]
alen= alen[['property_number','date','ngrantee1']]
fclose=fclose[['parcel','filedate','cdesig']]

In [90]:
##Parsing foreclosure to identify case designation of type "Board of Revision"

def parse_fc(x):
    if isinstance(x,str):
        if 'board of revision' in x.lower():
            return 'bor'
        else: 
            return 'other'
    else:
        return 'other'
fclose['case_desig']=fclose.cdesig.apply(parse_fc)

In [93]:
# Renaming columns for coherence and ascertaining unduplicated data
transfer= transfer.rename(columns={'property_number':'parcel','mdate':'date'})
alen= alen.rename(columns={'property_number':'parcel'})
fclose= fclose.rename(columns={'filedate':'date'})
transfer.drop_duplicates(inplace=True)
alen.drop_duplicates(inplace=True)
fclose.drop_duplicates(inplace=True)

In [94]:
import re
reo_owned=['BANK','MORTG','MORT.','FANNIE','SEC.','LOAN','AMERIQUEST','AMTRUST','ARGENT','AEON','BINARKO','CENTEX','CHASE'\
           ,'GMAC','HOMECOMINGS','INTERBAY','FEDERAL','NATIONAL','FREDDIE MAC','MORTGAGE','HOME LOAN MORTGAGE','HUD','VETERAN']
reo_owned=[x.lower() for x in reo_owned]

investor_owned=['LLC','LLLC','L.L.C','COMP','HOUSING','INC','OHIO','CLEVELAND','ESTATE','ORGANIZATION','DEVELOP','LTD','L.T.D.'\
                ,'HSG','LIMITED','CUYAHOGA','PROPERT','INVEST','REALT','HOMES','NEIGHBOR','LAND','AVENUE','ROAD','STREET','DRIVE',\
                'GROUP','CAPITAL','CONSTRUCTION','REHAB','RENTAL','REALTY','EVERGREEN']
investor_owned=[x.lower() for x in investor_owned]


def get_owner(x):
    if isinstance(x,str):
        s=x.lower()
        for name in  reo_owned:
            if name in s:
                return 'reo'
            else:
                for n in investor_owned:
                    if n in s:
                        return 'investor'
                    else: 
                        return 'other'
    else: pass
    
transfer['owner_type']=transfer.grantee1.apply(get_owner)
alen['owner_type']=alen.ngrantee1.apply(get_owner)

In [99]:
fclose['status']='fc'
fclose.drop('cdesig',axis=1,inplace=True)
transfer['status']='t'
alen['status']='al'
alen= alen.rename(columns={'ngrantee1':'grantee1'})
data= fclose.append(transfer[['parcel','date','status','owner_type','deed_type']]).append(alen[['parcel',\
    'date','status','owner_type']])

In [100]:
data.head()

,case_desig,date,deed_type,owner_type,parcel,status
0,other,2011-11-02,NaN,NaN,001-01-011,fc
1,other,2008-07-11,NaN,NaN,001-02-016,fc
2,other,2009-02-13,NaN,NaN,001-02-020,fc
3,other,2014-07-24,NaN,NaN,001-02-022,fc
4,other,2013-07-17,NaN,NaN,001-02-024,fc


In [103]:
def status_code(x):
    if x[5]=='fc' and x[0]=='other':
        return 1
    elif x[5]=='fc' and x[0]=='bor':
        return 2
    elif (x[5] == 't') and (x[3] == 'reo'):
        return 2
    elif (x[5] == 't') and (x[3] == 'investor'):
        return 3
    elif (x[5] == 'al') and (x[3] == 'reo'):
        return 2
    elif (x[5] == 'al') and (x[3] == 'investor'):
        return 3
    else:
        return 0

data['status_code']=data.apply(status_code,1)

In [104]:
data[data.status_code==2]

,case_desig,date,deed_type,owner_type,parcel,status,status_code
114,bor,2011-07-01,NaN,NaN,001-07-006,fc,2
133,bor,2008-05-15,NaN,NaN,001-07-051,fc,2
137,bor,2014-03-05,NaN,NaN,001-07-051,fc,2
181,bor,2011-08-10,NaN,NaN,001-07-071,fc,2
211,bor,2007-08-21,NaN,NaN,001-08-059,fc,2
215,bor,2014-09-18,NaN,NaN,001-08-059,fc,2
223,bor,2007-09-05,NaN,NaN,001-08-069,fc,2
232,bor,2007-01-12,NaN,NaN,001-08-069,fc,2
241,bor,2013-09-16,NaN,NaN,001-08-069,fc,2
281,bor,2013-09-25,NaN,NaN,001-08-141,fc,2


In [105]:
offset=pd.DateOffset(months=3)
rng=pd.date_range('2010/10/1',periods=21,freq='3MS')
data_final=pd.DataFrame(tci['parcel'])


for date in rng:
    subdata= data.loc[data.date <= date,].groupby('parcel').nth(-1)
    subdata=subdata[['status_code']].reset_index()
    data_final=pd.merge(data_final,subdata,left_on='parcel',right_on='parcel', how='left')
    data_final.rename(columns={'status_code':date})
    
data_final.columns=['parcel'] + [str(x) for x in rng]
data_final.columns=data_final.columns.str.replace(' 00:00:00','')


data_final.fillna(value=0,inplace=True)
    

In [106]:
data_final.head()

,parcel,2010-10-01,2011-01-01,2011-04-01,2011-07-01,2011-10-01,2012-01-01,2012-04-01,2012-07-01,2012-10-01,...,2013-07-01,2013-10-01,2014-01-01,2014-04-01,2014-07-01,2014-10-01,2015-01-01,2015-04-01,2015-07-01,2015-10-01
0,001-01-001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,001-01-003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,001-01-004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,001-01-005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,001-01-006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [107]:
data_final.to_csv('Z:/largetransfer/luc/carter/data/Quartely_parcel_status2.csv')
